In [ ]:
# Author: Fawaaz Kamali Siddiqui
# Date: 15 November 2025

import pandas as pd
import numpy as np
import re
from word2number import w2n

# Utility functions

# Standardize column names
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(".", "_")
        .str.replace("__", "_")
    )
    return df

# extract only numeric values from entries
def extract_numeric(value):
    """
    Examples: "305mm" → 305 ; "45 kg" → 45 ; "forty" → 40 ; "" → NaN.
    """
    if pd.isna(value):
        return np.nan

    value = str(value).strip().lower()

    # Try converting written numbers ("forty", "one hundred")
    try:
        return w2n.word_to_num(value)
    except:
        pass

    # Extract digits + decimal point
    numeric_match = re.findall(r"[-+]?\d*\.?\d+", value)
    if numeric_match:
        try:
            return float(numeric_match[0])
        except:
            return np.nan

    return np.nan


# convert F to C
def fahrenheit_to_celsius(value):
    """Convert F → C only when a trailing 'f' exists."""
    if pd.isna(value):
        return np.nan
    v = str(value).strip().lower()
    if v.endswith("f"):
        num = extract_numeric(v)
        return (num - 32) * 5/9
    else:
        return extract_numeric(v)

# convert in to mm
def inches_to_mm(value):
    """Convert inches → mm only when trailing 'in' exists."""
    if pd.isna(value):
        return np.nan
    v = str(value).strip().lower()
    if v.endswith("in"):
        num = extract_numeric(v)
        return num * 25.4     # 1 inch = 25.4 mm
    else:
        return extract_numeric(v)
    
# if negative values are found, replace with NaN
def enforce_non_negative(df, cols):
    """Replace negative values with NaN."""
    for c in cols:
        df[c] = df[c].apply(lambda x: x if pd.isna(x) or x >= 0 else np.nan)

# if ph values are outside 0-14, replace with NaN
def enforce_ph_range(df, col):
    """Ensure pH is between 0 and 14."""
    df[col] = df[col].apply(lambda x: x if pd.notna(x) and 0 <= x <= 14 else np.nan)

# enforce yield range with maximum yield of 200 t/ha
def enforce_yield_range(df, col="yield_t_ha", max_yield=200):
    """
    Enforce physical limits on yield_t_ha.
    Valid range: 0 to max_yield (default = 200 t/ha).
    Anything outside → NaN (junk).
    """
    df[col] = df[col].apply(
        lambda x: x if pd.notna(x) and 0 <= x <= max_yield else np.nan
    )

# Load datasets
crop_profiles = "../data/raw/crop_profiles_data.csv"
current_conditions = "../data/raw/current_conditions_data.csv"
historical_data = "../data/raw/field_historical_data.csv"

crop_profiles_df = pd.read_csv(crop_profiles)
current_conditions_df = pd.read_csv(current_conditions)
historical_data_df = pd.read_csv(historical_data)

# Clean column names
crop_profiles_df = clean_columns(crop_profiles_df)
current_conditions_df = clean_columns(current_conditions_df)
historical_data_df = clean_columns(historical_data_df)

# Clean Crop Profiles Dataset
crop_profiles_df["crop"] = crop_profiles_df["crop"].str.strip().str.lower()
crop_profiles_df["preferred_soil_type"] = crop_profiles_df["preferred_soil_type"].str.strip().str.lower()

# Columns to clean
numeric_cols_cp = [
    "ph_min", "ph_max", "water_need_mm",
    "nitrogen_need_kg_ha", "phosphorus_need_kg_ha",
    "potassium_need_kg_ha", "maturity_days"
]

for col in numeric_cols_cp:
    if col in crop_profiles_df.columns:
        crop_profiles_df[col] = crop_profiles_df[col].apply(extract_numeric)

# pH range enforcement
enforce_ph_range(crop_profiles_df, "ph_min")
enforce_ph_range(crop_profiles_df, "ph_max")

# No negatives allowed
enforce_non_negative(crop_profiles_df, numeric_cols_cp)

'''
# Clean Current Conditions Dataset
current_conditions_df["field_id"] = current_conditions_df["field_id"].str.strip().str.lower()
current_conditions_df["soil_type"] = current_conditions_df["soil_type"].str.strip().str.lower()
current_conditions_df["water_source"] = current_conditions_df["water_source"].str.strip().str.lower()

numeric_cols_cp = [
    "soil_ph", "soil_moisture_pct", "nitrogen_kg_ha",
    "phosphorus_kg_ha", "potassium_kg_ha", "forecast_rain_mm_next30d", 
    "avg_temp_next30d", "precip_history_mm_past90d"
]

for col in numeric_cols_cp:
    if col in current_conditions_df.columns:
        current_conditions_df[col] = current_conditions_df[col].apply(extract_numeric)

# pH range enforcement
enforce_ph_range(current_conditions_df, "soil_ph")
enforce_non_negative(current_conditions_df, numeric_cols_cp)
enforce_yield_range(current_conditions_df, "yield_t_ha", 200)


# Clean Historical Data Dataset
historical_data_df["field_id"] = historical_data_df["field_id"].str.strip().str.lower()
historical_data_df["soil_type"] = historical_data_df["soil_type"].str.strip().str.lower()
historical_data_df["previous_crop"] = historical_data_df["previous_crop"].str.strip().str.lower()
historical_data_df["crop_planted"] = historical_data_df["crop_planted"].str.strip().str.lower()
historical_data_df["water_source"] = historical_data_df["water_source"].str.strip().str.lower()

numeric_cols_hd = [
    "season_year", "soil_ph", "soil_moisture_pct", "nitrogen_kg_ha",
    "phosphorus_kg_ha", "potassium_kg_ha", "forecast_rain_mm", 
    "avg_temp", "precip_history_mm", "yield_t_ha"
]

for col in numeric_cols_hd:
    if col in historical_data_df.columns:
        historical_data_df[col] = historical_data_df[col].apply(extract_numeric)

# pH range enforcement
enforce_ph_range(historical_data_df, "soil_ph")
enforce_non_negative(historical_data_df, numeric_cols_hd)
enforce_yield_range(historical_data_df, "yield_t_ha", 200)


'''


# Preview
display(crop_profiles_df.head())
display(current_conditions_df.head())
display(historical_data_df.head())

display(crop_profiles_df.iloc[39])


,crop,preferred_soil_types,ph_min,ph_max,water_need_mm,nitrogen_need_kg_ha,phosphorus_need_kg_ha,potassium_need_kg_ha,maturity_days
0,wheat,"Loam,Peat,Clay,Sandy",5.75,7.68,381.0,106.0,44,68,105
1,corn,"Sandy,Loam",5.64,7.48,NaN,152.0,60,81,133
2,soybean,"Peat,Loam,Clay,Sandy",5.89,7.68,411.0,20.0,24,61,92
3,barley,Loam; Sandy,5.61,7.51,321.0,95.0,32,51,104
4,canola,"Peat,Loam,Clay,Silt",5.81,7.54,362.0,114.0,47,66,135


,field_id,soil_ph,soil_moisture_pct,nitrogen_kg_ha,phosphorus_kg_ha,potassium_kg_ha,forecast_rain_mm_next30d,avg_temp_next30d,precip_history_mm_past90d,soil_type,previous_crops_last_3_years,water_source
0,Field_001,5.82,54.2,20.1,11.0,67.8,263.8,27.4,508.3,Silt,"Pear, Carrot, Banana",canal
1,Field_002,7.60,50.3,68.3,20.3,55.7,138.0,19.6,2 in,Clay,"Melon, Garlic, Spinach",canal
2,Field_003,6.47,19,42.9,48.7,64.7,408.0,12,247.6,Loam,"Pepper, Pea, Soybean",canal
3,Field_004,7.11,36.8,23.1,38.3,56.6,226.0,22.1,118.9,Clay,"Beet, Mango, Canola",well
4,Field_005,6.72,31,77.3,17.2,88.1,506.4,17.2,277.2,Silt,"Cocoa, Cabbage, Bean",canal


,field_id,season_year,soil_ph,soil_type,soil_moisture_pct,nitrogen_kg_ha,phosphorus_kg_ha,potassium_kg_ha,forecast_rain_mm,avg_temp,precip_history_mm,previous_crop,crop_planted,yield_t_ha,water_source
0,Field_034,2022,7.53,loam,38.14,91.3,22.7,106.39,285.41,19.15,92.74,Sunflower,Kale,9.26,well
1,Field_007,2025,7.88,PEAT,21.28,54.38,45.83,55.26,forty,25.34,264.74,Sunflower,Blueberry,2.86,canal
2,Field_046,2025,8.28,SILT,10.49,147.1,16.84,53.85,475.32,36.66,89.19,Pepper,Broccoli,4.68,rainfed
3,Field_011,2020,5.41,CLAY,16.47,23.82,21.25,107.4,22.0 in,11.69,211.72,Cocoa,Cocoa,1.21,well
4,Field_011,2018,7.22,CLAY,19.35,25.29,23.49,113.46,586.29,10.06F,259.03,Onion,Onion,8.72,well


crop                     blueberry
preferred_soil_types          Loam
ph_min                        5.64
ph_max                         6.8
water_need_mm                306.0
nitrogen_need_kg_ha          103.0
phosphorus_need_kg_ha           52
potassium_need_kg_ha            88
maturity_days                  104
Name: 39, dtype: object